In [11]:
import pandas as pd

df = pd.read_csv('./Reports/газпром_3.csv',
                 sep=';',
                 encoding='utf-8',
                 header=0 )
unique_markers = df['Маркер'].unique().tolist()

c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df['Первый контакт'] = df['Первый контакт'].fillna('нет')
#df['Оператор'] = df['Оператор'].fillna('Нет оператора')

In [13]:
df

,№ п/п,ID звонка,Оператор,Дата звонка,Длительность звонка,"Всего пауз, сек","Максимальная продолжительность паузы, сек",Маркер,Маркер - количество совпадений,Первый контакт
0,1.0,mikopbx-1683000343,NaN,02.05.2023 09:08:11,00:02:24,9.8,1.3,🔠 12. Работа с неплательщиками,1.0,нет
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅Прощание,1.0,нет
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✔️ Прощание оператор,1.0,нет
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"❌Слова-паразиты, просторечия",2.0,нет
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1) Порядок оплаты,1.0,нет
...,...,...,...,...,...,...,...,...,...,...
230100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3) Разъяснение начисленных пени и госпошлины,1.0,нет
230101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,❔ Тест гипотезы: отсутствие уведомления,1.0,нет
230102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✔Перевод звонка,2.0,нет
230103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅Приветствие. Обязательные фразы,1.0,нет


In [14]:
# Forward fill NaN values in 'Маркер' column
df.fillna(method='ffill', inplace=True)

# Create a pivot table with 'Маркер' as columns and 'Маркер - количество совпадений' as values
pivot_df = df.pivot_table(index=['№ п/п', 'ID звонка', 'Оператор', 'Дата звонка', 'Длительность звонка', 'Всего пауз, сек', 'Первый контакт'], columns='Маркер', values='Маркер - количество совпадений').reset_index()

# Reset the index and rename the columns
pivot_df.columns.name = None  # Remove the columns' name

# Fill NaN values with 0 if needed
pivot_df = pivot_df.fillna(0)

# Now, 'pivot_df' contains the desired structure with 'Маркер' values as columns.



In [10]:
pivot_df.to_clipboard()

Работаем с получившейся простыней

In [22]:
# Обязательная часть
df = pivot_df
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
df['Дата'] = pd.to_datetime(df['Дата звонка'], format='%d.%m.%Y %H:%M:%S')
df['Дата'] = df['Дата'].dt.strftime('%d.%m.%Y')
df = df.reset_index(drop=True)

In [23]:
df['Первый контакт'] = df['Первый контакт'].replace('да', True).replace('нет', False)

In [27]:
# Свожу универсальную простыню с маркерами
pivot_df = df.pivot_table(index='Оператор', aggfunc='sum')

In [25]:
pivot_table = df.pivot_table(index='Оператор', values='Первый контакт', aggfunc='sum')

In [16]:
pivot_table = df.pivot_table(index='Оператор', values='Длительность звонка', aggfunc='sum')

In [29]:
pivot_df.to_clipboard()

In [20]:
pivot_table['Длительность звонка'] = pivot_table['Длительность звонка'].dt.total_seconds() / 3600
pivot_table

,Длительность звонка
Оператор,
Апанасенко Александра (19622),30.682778
Аторина Марина (19624),10.613889
Бобылева Наталья (19618),33.435556
Борищенкова Евгения (19642),23.348056
Булавина Елена (19623),49.693889
Высоцкая Анна (19628),8.029444
Дука Оксана (19634),38.251944
Ешакина Маргарита (19611),0.459444
Иванина Светлана (19629),32.341111


In [5]:
# Blocks dfs
blocks = [0, [], [], [], [], [], [], []]
for i in df.columns:
    for j in range(1, 8):
        if str(j) in i[:1]: 
            blocks[j].append(i)
# Politeness
products = [
    'ИИС+ПИФ',
    'ИСЖ',
    'Категория КК',
    'Моя карта ',
    'Мультибонус',
    'Вклады',
    'Вездедоход (КК)'

]
# Warning
negativity = [
'Слова-паразиты',
'Нарушение стандартов взаимодействия. Провоцирование конфликтов. ', 
'Нежелательные выражения',
'Нецензурная лексика']
# Start
politeness = [
'Соблюдение стандартов взаимодействия. Вежливость+эмпатия в процессе обслуживания (Стандарты)',
'Соблюдение стандартов взаимодействия. Положительные эмоции',
'Соблюдение стандартов взаимодействия. Эмпатия.',
'Эмпатия и Заинтересованность'
] 

In [6]:

# Remove "NaN" strings from the list
unique_markers = [x for x in unique_markers if type(x) is str]

In [7]:
#rf = df.pivot_table(index='Оператор', values=unique_markers, aggfunc='sum')
total_time_df = df.pivot_table(index='Оператор', values='Длительность звонка', aggfunc='sum')
# Convert timedelta to seconds and then to hours
total_time_df['Длительность звонка (часы)'] = total_time_df['Длительность звонка'].dt.total_seconds() / 3600
total_time_df.columns = ['Всего записи', 'Всего записи (часы)']
total_wait_df = df.pivot_table(index='Оператор', values='Всего пауз, сек', aggfunc='sum') / 3600
total_wait_df.columns = ['Всего пауз (часы)']


In [8]:
pivot_df

,1. Установка контакта. Приветствие,1.2 Установка контакта. Представление,1.3 Установка контакта. Уточнение имени клиента,1.4 Установка контакта. Уточнение цели клиента,1.5 Установка контакта. Определение профиля клиента,1.6 Установка контакта. Актуализация контактных данных,2. Выявление и формирование потребности. Сформировать потребность в ДК,2. Выявление и формирование потребности. Сформировать потребность в кредитной карте,2. Выявление потребностей ИСП,2.1 Выявление и формирование потребности. Сформировать потребность в КН,...,💳 Вклад,💳 Дебетовая карта,💳 ИИС,💳 ИСЖ,💳 НПФ,💳 ПЛ «Мультибонус»,💳 Потребительский кредит + КК,💳 Сберегательный счет,🔹 Завершение диалога,🔹 Начало диалога
Оператор,,,,,,,,,,,,,,,,,,,,,
Бохан Инна,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Валгуцкова Наталья,15.0,34.0,0.0,1002.0,5.0,2.0,45.0,78.0,15.0,81.0,...,189.0,82.0,12.0,13.0,0.0,3.0,224.0,21.0,93.0,421.0
Варнавина Марина,11.0,91.0,2.0,1634.0,7.0,4.0,88.0,73.0,4.0,30.0,...,273.0,326.0,6.0,0.0,0.0,62.0,156.0,165.0,395.0,837.0
Гугнина Валентина,0.0,6.0,0.0,42.0,0.0,0.0,4.0,3.0,0.0,1.0,...,0.0,11.0,0.0,0.0,0.0,0.0,3.0,6.0,9.0,36.0
Давтян Мариам,9.0,18.0,0.0,392.0,3.0,5.0,9.0,35.0,4.0,27.0,...,140.0,46.0,1.0,7.0,0.0,3.0,78.0,30.0,31.0,156.0
Евтеев Руслан,2.0,2.0,1.0,224.0,1.0,1.0,53.0,67.0,5.0,16.0,...,61.0,53.0,2.0,5.0,1.0,1.0,75.0,57.0,27.0,205.0
Еженкова Виктория,0.0,2.0,0.0,20.0,0.0,0.0,1.0,14.0,0.0,2.0,...,7.0,0.0,0.0,0.0,0.0,0.0,13.0,3.0,1.0,27.0
Кирия Татьяна,1.0,2.0,0.0,259.0,0.0,0.0,25.0,23.0,4.0,26.0,...,68.0,76.0,6.0,2.0,0.0,0.0,44.0,29.0,10.0,153.0
Кичук Павел,4.0,48.0,1.0,349.0,11.0,1.0,33.0,32.0,3.0,46.0,...,139.0,93.0,5.0,9.0,1.0,2.0,61.0,34.0,65.0,263.0


In [9]:
# TAB 1 SUMMARY
block_1 = pivot_df[blocks[1]].sum(axis=1)
block_1.name = 'Блок 1: Установка контакта'
block_2 = pivot_df[blocks[2]].sum(axis=1)
block_2.name = 'Блок 2: Выявление и формирование потребности'
block_3 = pivot_df[blocks[3]].sum(axis=1)
block_3.name = 'Блок 3: Презентация основного продукта'
block_4 = pivot_df[blocks[4]].sum(axis=1)
block_4.name = 'Блок 4: Работа с возражениями'
block_5 = pivot_df[blocks[5]].sum(axis=1)
block_5.name = 'Блок 5: Завершение продажи'
block_6 = pivot_df[blocks[6]].sum(axis=1)
block_6.name = 'Блок 6: Кросс- продажа'
block_7 = pivot_df[blocks[7]].sum(axis=1)
block_7.name = 'Блок 7: Завершение встречи'
script_df = pd.concat([block_1, block_2, block_3, block_4, block_5, block_6, block_7], axis=1)
# TAB 2 ПРОДУКТЫ
products_df = pivot_df[products]
# TAB 2 НЕГАТИВ
negative_df = pivot_df[negativity]
# TAB 3 Вежливость
politeness_df = pivot_df[politeness]

# TAB 0 Свод
summary_df = pd.concat([total_time_df, total_wait_df], axis=1)
summary_df['К активности'] = 1 - (summary_df['Всего пауз (часы)'] / summary_df['Всего записи (часы)'])
k_products = block_7 / summary_df['Всего записи (часы)']
k_products.name = 'К кросс-продаж*'
k_conformity = script_df.sum(axis=1) / summary_df['Всего записи (часы)']
k_conformity.name = 'К соответсвия**'
k_negativity = negative_df.drop(columns=['Слова-паразиты']).sum(axis=1) / summary_df['Всего записи (часы)']
k_negativity.name = 'К негатива***'
summary_df = pd.concat([summary_df, k_products, k_conformity, k_negativity], axis=1)
del summary_df['Всего записи']

In [10]:
summary_df

,Всего записи (часы),Всего пауз (часы),К активности,К кросс-продаж*,К соответсвия**,К негатива***
Оператор,,,,,,
Бохан Инна,0.080000,0.008028,0.899653,0.000000,12.500000,0.000000
Валгуцкова Наталья,87.961667,33.649583,0.617452,4.138166,36.163480,4.035849
Варнавина Марина,153.793333,63.432778,0.587545,5.182279,31.789414,1.228922
Гугнина Валентина,31.941111,17.529167,0.451204,0.970536,4.132605,0.250461
Давтян Мариам,105.269722,49.067278,0.533890,1.320418,12.396727,0.902444
Евтеев Руслан,110.748056,73.682417,0.334684,1.110629,8.876002,0.361180
Еженкова Виктория,23.793889,15.026694,0.368464,0.378248,2.605711,0.252166
Кирия Татьяна,49.352778,26.107472,0.471003,4.538752,21.457759,1.175212
Кичук Павел,47.136111,22.658278,0.519301,5.325004,30.167953,1.866934


In [11]:
negative_df

,Слова-паразиты,Нарушение стандартов взаимодействия. Провоцирование конфликтов.,Нежелательные выражения,Нецензурная лексика
Оператор,,,,
Бохан Инна,23.0,0.0,0.0,0.0
Валгуцкова Наталья,9976.0,39.0,267.0,49.0
Варнавина Марина,9621.0,29.0,152.0,8.0
Гугнина Валентина,204.0,1.0,7.0,0.0
Давтян Мариам,3538.0,10.0,77.0,8.0
Евтеев Руслан,2252.0,3.0,35.0,2.0
Еженкова Виктория,92.0,1.0,5.0,0.0
Кирия Татьяна,2508.0,3.0,51.0,4.0
Кичук Павел,3334.0,7.0,69.0,12.0


In [12]:
color_scale_rule_percent = {
                'type': '2_color_scale',
                'min_color': '#FFFFFF',  # White
                'max_color': '#8EA9DB',  # Red
                'min_type': 'percentile',
                'min_value': 0,
                'max_type': 'percentile',
                'max_value': 100
                }

with pd.ExcelWriter('pochta_bank.xlsx', engine='xlsxwriter') as writer:
    def create_sheet(pivot_table, sheet_name, summary=False):
        # Write the DataFrame to the Excel file
        pivot_table.to_excel(writer, sheet_name=sheet_name)
        # Access the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        white_fill_format = workbook.add_format({'text_wrap': False, 'bg_color': '#FFFFFF', 'border': 0})
        bold_format = workbook.add_format({'italic': True, 'text_wrap': False, 'bg_color': '#FFFFFF', 'border': 0})
        worksheet.set_column(0, 200, 20, white_fill_format)
        for i in range(1, pivot_df.shape[1]):
            worksheet.conditional_format(1, i, 999, i, color_scale_rule_percent)
        if summary:
            worksheet.write(f'A{pivot_df.shape[0]+3}', '* - кол-во кросс-продаж (согласно маркерам) в час', bold_format)
            worksheet.write(f'A{pivot_df.shape[0]+4}', '** - кол-во ключевых фраз из методологических материалов (согласно маркерам) в час', bold_format)
            worksheet.write(f'A{pivot_df.shape[0]+5}', '*** - кол-во маркеров негатива в час', bold_format)

            

    create_sheet(summary_df, 'Сводный отчет', summary=True)        
    create_sheet(script_df, 'Следование скрипту')
    create_sheet(products_df, 'Продукты банка')
    create_sheet(negative_df, 'Негатив')
    create_sheet(politeness_df, 'Вежливость')
    create_sheet(pivot_df, 'Все маркеры')
        

In [59]:
pivot_df.shape

(18, 118)

# Почта Банк
- Проверка точности

In [1]:
import pandas as pd

In [2]:
topic1 = [['Причина1','Причина2'],['Серый негатив','Серый негатив'],['Качество работы Сети','Офисы. Сеть офисов'],['Цена. Кредит','Кредит Гарантированная ставка'],['Перевод денежных средств','Сроки зачисления. Пенсия/пособия'],['Банкоматы','Мало банкоматов'],['Цена. Кредит','Кредит Процентная ставка'],['Перевод денежных средств','Процесс перевода и платежей. Другое'],['Качество. Дебетовая карта','Дебетовая карта Общие условия и тарифы'],['Цена. Кредит','Кредит Рефинансирование'],['Серый позитив','Серый позитив'],['Банкоматы','Снятие и внесение наличных'],['ДБО','Функциональность и удобство. Общий функционал'],['Качество. Кредит','Кредит Одобрение продукта'],['SMS/PUSH уведомления','Качество работы уведомлений'],['Качество работы Сети','Сотрудники. Вежливость'],['Качество. Вклад','Вклад Общие условия и тарифы'],['Общее про Банк','Репутация'],['Общее про Банк','Надежность и стабильность'],['Цена. Текущие счета','Текущие счета. Процент на остаток'],['Качество работы Сети','Общая оценка офисов'],['Качество работы Сети','Офисы. Очереди'],['Качество работы Сети','Сотрудники. Компетентность'],['Банкоматы','Сеть банкоматов'],['Перевод денежных средств','Сроки зачисления. Другое'],['SMS/PUSH уведомления','Навязчивость уведомлений'],['Реклама','Не соотвеТекущие счетатвует рекламе'],['Цена. Дебетовая карта','Дебетовая карта Программы лояльности'],['ДБО','Функциональность и удобство. Коммунальные платежи'],['Цена. Текущие счета','Текущие счета. Комиссия за перевод'],['Качество работы Клиентской службы','Служба поддержки. Вежливость'],['Качество работы телефонных продаж','Продажи. Навязчивость'],['Качество. Кредит','Кредит Погашение и досрочное погашение'],['Качество. Кредит','Кредит Общие условия и тарифы'],['Качество. Дебетовая карта','Дебетовая карта Оформление продукта'],['Качество работы Службы по взысканию задолженности','СпВЗ. Навязчивость'],['ДБО','Функциональность и удобство. Навигация'],['Качество работы Сети','Офисы. Режим работы'],['ДБО','Общее отношение к МП/ЛК'],['Банкоматы','Качество работы. Отсутствуют наличные'],['Цена. Вклад','Вклад Процентная ставка'],['Качество работы Клиентской службы','Служба поддержки. Компетентность'],['Общее про Банк','Популярность'],['Банкоматы','Переводы и платежи'],['Цена. Кредит','Кредит Страховка'],['ДБО','Функциональность и удобство. Платежи и переводы'],['ДБО','Стабильность работы. Проблемы с входом'],['Цена. Дебетовая карта','Дебетовая карта Выпуск и обслуживание'],['Банкоматы','Качество работы. Зависание'],['Качество. Текущие счета','Текущие счета. Общие условия и тарифы'],['Качество работы Клиентской службы','Время дозвона'],['Качество. Вклад','Вклад Оформление продукта'],['Качество работы Службы по взысканию задолженности','СпВЗ. Вежливость'],['Качество работы Сети','Сотрудники. Навязывание продуктов'],['Реклама','Навязчивая реклама'],['Качество работы Сети',    'Сотрудники. Отсутствие на рабочем месте']]

In [3]:
df = pd.read_csv('pochta_nps2.csv', header=0, sep=';')

In [4]:
df['ID звонка'] = df['ID звонка'].fillna(method='ffill')

In [5]:
df['Основная тема'] = ''

,№ п/п,ID звонка,Длительность звонка,Маркер,Оценка,Основная тема
0,1.0,cpl-70-1695022902414647492_589798789,00:01:36,📁 Офисы. Очереди,NaN,
1,NaN,cpl-70-1695022902414647492_589798789,NaN,📁 Серый позитив,NaN,
2,2.0,cpl-70-1695023019402856108_1062443267,00:02:41,📁 Серый позитив,NaN,
3,3.0,cpl-70-1695023135385742513_589940229,00:01:42,NaN,NaN,
4,4.0,cpl-70-1695023559413502426_1062979331,00:01:34,📁 Дебетовая карта Программы лояльности,NaN,
...,...,...,...,...,...,...
1789,NaN,cpl-71-1695113057428931541_629165572,NaN,📁 Серый позитив,NaN,
1790,997.0,cpl-71-1695113362369178328_1287670402,00:02:10,📁 Серый позитив,NaN,
1791,998.0,cpl-71-1695113376426906425_1287673730,00:01:45,📁 Серый позитив,NaN,
1792,999.0,cpl-71-1695113410398219937_1287686658,00:01:42,NaN,NaN,


In [6]:
for i in range(0, df.shape[0]):
    # Вернуть первый топик
    topic2 = df['Маркер'].iloc[i]
    print(type(topic2))
    for j in topic1:
        if type(topic2) is float:
            pass
        elif j[1] in topic2:
            print(j[1], topic2)
            print(f'Matched Т2:{topic2} ---> Т1:{j[0]}')
            df['Основная тема'].iloc[i] = j[0]


    #df['Основная тема'].iloc[i] = i

<class 'str'>
Офисы. Очереди 📁 Офисы. Очереди
Matched Т2:📁 Офисы. Очереди ---> Т1:Качество работы Сети
<class 'str'>
Серый позитив 📁 Серый позитив
Matched Т2:📁 Серый позитив ---> Т1:Серый позитив
<class 'str'>
Серый позитив 📁 Серый позитив
Matched Т2:📁 Серый позитив ---> Т1:Серый позитив
<class 'float'>
<class 'str'>
Дебетовая карта Программы лояльности 📁 Дебетовая карта Программы лояльности
Matched Т2:📁 Дебетовая карта Программы лояльности ---> Т1:Цена. Дебетовая карта
<class 'str'>
Серый позитив 📁 Серый позитив
Matched Т2:📁 Серый позитив ---> Т1:Серый позитив
<class 'str'>
Офисы. Сеть офисов 📁 Офисы. Сеть офисов
Matched Т2:📁 Офисы. Сеть офисов ---> Т1:Качество работы Сети
<class 'str'>
Качество работы. Зависание 📁 Качество работы. Зависание
Matched Т2:📁 Качество работы. Зависание ---> Т1:Банкоматы
<class 'str'>
Офисы. Очереди 📁 Офисы. Очереди
Matched Т2:📁 Офисы. Очереди ---> Т1:Качество работы Сети
<class 'str'>
Серый позитив 📁 Серый позитив
Matched Т2:📁 Серый позитив ---> Т1:Серый п

c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


<class 'str'>
Продажи. Навязчивость 📁 Продажи. Навязчивость
Matched Т2:📁 Продажи. Навязчивость ---> Т1:Качество работы телефонных продаж
<class 'str'>
Офисы. Сеть офисов 📁 Офисы. Сеть офисов
Matched Т2:📁 Офисы. Сеть офисов ---> Т1:Качество работы Сети
<class 'str'>
Сеть банкоматов 📁 Сеть банкоматов
Matched Т2:📁 Сеть банкоматов ---> Т1:Банкоматы
<class 'str'>
Снятие и внесение наличных 📁 Снятие и внесение наличных
Matched Т2:📁 Снятие и внесение наличных ---> Т1:Банкоматы
<class 'str'>
Серый позитив 📁 Серый позитив
Matched Т2:📁 Серый позитив ---> Т1:Серый позитив
<class 'str'>
Офисы. Сеть офисов 📁 Офисы. Сеть офисов
Matched Т2:📁 Офисы. Сеть офисов ---> Т1:Качество работы Сети
<class 'str'>
Общее отношение к МП/ЛК 📁 Общее отношение к МП/ЛК
Matched Т2:📁 Общее отношение к МП/ЛК ---> Т1:ДБО
<class 'str'>
Серый позитив 📁 Серый позитив
Matched Т2:📁 Серый позитив ---> Т1:Серый позитив
<class 'str'>
Функциональность и удобство. Общий функционал 📁 Функциональность и удобство. Общий функционал
Mat

In [7]:
df['Подтема'] = df['Маркер'].str[2:]


In [9]:
df.to_clipboard()

# 2C transformer 1

In [14]:
import pandas as pd

csv_list = ['2С_matrix.csv']

df = pd.DataFrame()
for i in csv_list:
    '''
    Take report files 1-by-1 and the merge then on external index from indices.py
    This will cut RAM cost 30 times (and make shit slower)
    '''
    # Merge 2 frames
    df_add = pd.read_csv(i, sep=';', encoding='utf-8',header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Forward fill NaN values in 'Маркер' column
df.fillna(method='ffill', inplace=True)

# OPTION: DROP ALL NOT CONTAINING
df = df[df['Маркер'].str.contains('🦝')]


df = df.fillna(0)
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
df['Дата'] = pd.to_datetime(df['Дата звонка'], format='%d.%m.%Y %H:%M:%S')
df['Дата'] = df['Дата'].dt.strftime('%d.%m.%Y')
df = df.reset_index(drop=True)
print(df)
# Create a pivot table with 'Маркер' as columns and 'Маркер - количество совпадений' as values
index_cols = [col for col in df.columns if col not in ['Маркер', 'Маркер - количество совпадений']]
pivot_df = df.pivot_table(
    index=index_cols,
    columns='Маркер',
    values='Маркер - количество совпадений').reset_index()
# Reset the index and rename the columns
#pivot_df.columns.name = None  # Remove the columns' name


       № п/п          Дата звонка 2C: Результаты обзвона 2C: Контактные лица  \
0        2.0  29.11.2023 08:01:05       Связь прервалась                   0   
1        3.0  29.11.2023 08:01:34       Связь прервалась                   0   
2        5.0  29.11.2023 08:01:50   Передадут информацию         Третьe лицо   
3        5.0  29.11.2023 08:01:50   Передадут информацию         Третьe лицо   
4        6.0  29.11.2023 08:01:51       Связь прервалась         Третьe лицо   
...      ...                  ...                    ...                 ...   
1460  1607.0  30.11.2023 12:47:35       Связь прервалась             Должник   
1461  1629.0  30.11.2023 13:36:39       Связь прервалась         Третьe лицо   
1462  1630.0  30.11.2023 13:44:46       Связь прервалась         Третьe лицо   
1463  1631.0  30.11.2023 13:46:37       Связь прервалась         Третьe лицо   
1464  1634.0  30.11.2023 13:48:16       Связь прервалась         Третьe лицо   

     Длительность звонка  Номер телефон

In [15]:
pivot_df.to_excel('2c_test.xlsx')

In [16]:
pivot_df

Маркер,№ п/п,Дата звонка,2C: Результаты обзвона,2C: Контактные лица,Длительность звонка,Номер телефона спикера,2С: Номер дела,2C: Клиенты,Тип звонка,Инициатор завершения,...,🦝 АО Целый день звонки (жф),"🦝 АО Я не должник, с какой вы организации (сл)",🦝 АО ассистент,🦝 Автоинформатор / Умные АО,🦝 Автоинформатор / Умные АО 2,🦝 Автоинформатор / Умные АО 3,🦝 Автоответчик ДР,🦝 Автоответчик Олег,🦝 Автоответчик организации,🦝 Автоответчик организации 2
0,2.0,29.11.2023 08:01:05,Связь прервалась,0,0 days 00:00:33,8.932424e+10,2350961,Сбербанк,Исходящий - АИ (интеллектуальный),Окончен оператором,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,29.11.2023 08:01:34,Связь прервалась,0,0 days 00:00:38,8.911746e+10,119096742,Сбербанк,Исходящий - АИ (интеллектуальный),Окончен оператором,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,29.11.2023 08:01:50,Передадут информацию,Третьe лицо,0 days 00:01:19,8.383225e+10,0441-P-9059140600,Сбербанк,Исходящий - АИ (интеллектуальный),0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
3,6.0,29.11.2023 08:01:51,Связь прервалась,Третьe лицо,0 days 00:00:11,8.909054e+10,938758,Сбербанк,Исходящий - АИ (интеллектуальный),Окончен оператором,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,7.0,29.11.2023 08:02:18,Связь прервалась,Третьe лицо,0 days 00:03:58,8.960072e+10,0393-P-15476518720,Сбербанк,Исходящий - АИ (интеллектуальный),Окончен оператором,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,1607.0,30.11.2023 12:47:35,Связь прервалась,Должник,0 days 00:00:52,8.926847e+10,KRS048391/008/20,ООО АБК,Исходящий - АИ (интеллектуальный),Окончен оператором,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1119,1629.0,30.11.2023 13:36:39,Связь прервалась,Третьe лицо,0 days 00:00:43,8.927436e+10,500418,Сбербанк,Исходящий - АИ (интеллектуальный),0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1120,1630.0,30.11.2023 13:44:46,Связь прервалась,Третьe лицо,0 days 00:00:36,8.927945e+10,1203-P-5662218330,Сбербанк,Исходящий - АИ (интеллектуальный),Окончен оператором,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1121,1631.0,30.11.2023 13:46:37,Связь прервалась,Третьe лицо,0 days 00:00:51,8.920445e+10,16ТКПР22121500125854,Сбербанк,Исходящий - АИ (интеллектуальный),Окончен оператором,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
